### **Funciones**

Este Notebook tiene como objetivo crear pequeños DataSets a partir de los datos procesados, con la finalidad de optimizar el trabajo de las funciones y evitar problemas de memoria en render.

#### **Librerias**

In [73]:
import pandas as pd

#### **Extracción y lectura de datos preparados**

In [140]:
# lectura de archivos 
df_steam_games=pd.read_csv('ETL/steam_games.csv')
df_user_items=pd.read_csv('ETL/user_items.csv')
df_user_reviews=pd.read_csv('ETL/user_reviews.csv')

In [141]:
df_steam_games.head(3)

,publisher,genero,title,fecha_lanzamiento,price,early_access,id_juego,developer,año_lanzamiento
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018


In [142]:
df_user_items.head(3)

,user_id,items_count,id_juego,nombre_juego,timpo_total_jugado,tiempo_jugado_ultimas_2semanas
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0


In [143]:
df_user_items.head(3)

,user_id,items_count,id_juego,nombre_juego,timpo_total_jugado,tiempo_jugado_ultimas_2semanas
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0


##### **Funcion 1** ______________________________________________________________________________________________________________________________________________________________________________

In [144]:
df_funcion1=pd.merge(df_user_items,df_steam_games,on='id_juego',how='left')

In [ ]:
df_funcion1=df_funcion1.loc[:,['genero','año_lanzamiento','timpo_total_jugado']].groupby(by=['genero','año_lanzamiento']).sum()


In [150]:
df_funcion1.reset_index(inplace=True)

In [152]:
df_funcion1.shape

(322, 3)

In [153]:
df_funcion1.isnull().sum()

genero                0
año_lanzamiento       0
timpo_total_jugado    0
dtype: int64

In [154]:
df_funcion1['genero'].unique()

array(['Action', 'Adventure', 'Animation  Modeling', 'Audio Production',
       'Casual', 'Design  Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Software Training', 'Sports',
       'Strategy', 'Utilities', 'Video Production', 'Web Publishing'],
      dtype=object)

In [80]:
# limpiar los siguientes tipos de datos 
# Design &amp; Illustration',   Animation &amp; Modeling 

In [155]:
# Creo un Dataset reducido con los datos necesarios para ser consumido por la API 
df_funcion1.to_csv('funcion1.csv',index=False) 

In [82]:
def PlayTimeGenre( genre : str ): #  Debe devolver año con mas horas jugadas para dicho género.
    # pasar a dataframe dentro de la funcion ?
    try:
        valor_maximo=df_funcion1[df_funcion1['genero']==genre]['timpo_total_jugado'].max()
        indice=df_funcion1[df_funcion1['timpo_total_jugado']==valor_maximo].index
        resultado=df_funcion1['año_lanzamiento'].loc[indice].values
        return {f'Año de lanzamiento con mas horas jugadas para el Género {genre}:':resultado[0]}
    except Exception as e:
        print('Genero incorrecto')


# Ejemplo de retorno: {"Año de lanzamiento con más horas jugadas para Género X" : 2013}

# Como unir las tablas ? 
    # usar codigo SQL
    # pandas con merge 
    # pandas con join 

In [83]:
PlayTimeGenre('Web Publishing')

Genero incorrecto


##### **Funcion 2** ______________________________________________________________________________________________________________________________________________________________________________

In [156]:
# elegir el dataframe reducido a utilizar

df_steam_games.head(3) # Exploracion 

,publisher,genero,title,fecha_lanzamiento,price,early_access,id_juego,developer,año_lanzamiento
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018


In [157]:
df_user_items.head(3) # Exploracion

,user_id,items_count,id_juego,nombre_juego,timpo_total_jugado,tiempo_jugado_ultimas_2semanas
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0


In [158]:
df_user_reviews.head(3) # Exploracion 

,user_id,id_juego,helpful,recommend,sentiment_analysis,review_posted,año_posted_review
0,76561197970982479,1250,No ratings yet,True,2,2011-11-05,2011
1,76561197970982479,22200,No ratings yet,True,2,2011-07-15,2011
2,76561197970982479,43110,No ratings yet,True,2,2011-04-21,2011


In [159]:
df_funcion2=pd.merge(df_user_items,df_steam_games,on='id_juego',how='left').merge(df_user_reviews,on=['user_id','id_juego'],how='left')

In [160]:
df_funcion2=df_funcion2.loc[:,['genero','user_id','año_posted_review','timpo_total_jugado']] # seleccionar columnas que necesito para la funcion

In [161]:
# Ver valores nulos de cada columna 
df_funcion2.isnull().sum()

#   Desicion: dado que necesito obligatoriamente el dato de "genero" y "año", desido eliminar los nulos en las mencionadas columnas, se observa que para algunos juegos no existe ni genero ni año_posted_Review
# lo cual indica que hay jugadores que han jugado un juego pero no han hecho una review del mismo 

genero                 1103548
user_id                      0
año_posted_review     10512840
timpo_total_jugado           0
dtype: int64

In [164]:
df_funcion2.shape

(77429, 4)

In [163]:
# elimino nulos 
df_funcion2.dropna(inplace=True)

In [165]:
df_funcion2.groupby(by=['genero','user_id','año_posted_review']).sum().reset_index(inplace=True)

In [166]:
df_funcion2  # df definitivo 

,genero,user_id,año_posted_review,timpo_total_jugado
50,Action,76561197970982479,2011.0,271
51,Indie,76561197970982479,2011.0,271
55,Action,76561197970982479,2011.0,10006
1003,Indie,js41637,2013.0,551
1004,Simulation,js41637,2013.0,551
...,...,...,...,...
10584623,Action,76561198239215706,2015.0,4659
10585465,Action,wayfeng,2015.0,42740
10585602,RPG,76561198251004808,2015.0,1098
10586733,Action,72947282842,2015.0,33


In [167]:
df_funcion2.to_csv('funcion2.csv',index=False)

In [94]:
df_funcion2

,genero,user_id,año_posted_review,timpo_total_jugado


In [168]:
df_funcion2[df_funcion2['genero']=='Action']

,genero,user_id,año_posted_review,timpo_total_jugado
50,Action,76561197970982479,2011.0,271
55,Action,76561197970982479,2011.0,10006
2356,Action,evcentric,2014.0,3225
2435,Action,evcentric,2014.0,447
2463,Action,evcentric,2014.0,508
...,...,...,...,...
10584396,Action,76561198236893796,2015.0,1078
10584623,Action,76561198239215706,2015.0,4659
10585465,Action,wayfeng,2015.0,42740
10586733,Action,72947282842,2015.0,33


In [169]:

valor_maximo=df_funcion2['timpo_total_jugado'].max()
indice_tiempo_maximo=df_funcion2[df_funcion2['timpo_total_jugado']==valor_maximo].index

In [170]:
indice_tiempo_maximo

Index([1250897, 1250898], dtype='int64')

In [171]:
df_funcion2['user_id'].loc[indice_tiempo_maximo].values

array(['wolop', 'wolop'], dtype=object)

In [172]:
df_genero=df_funcion2[df_funcion2['genero']=='Action']
tiempo_maximo=df_genero['timpo_total_jugado'].max()
indice_tiempo_maximo=df_genero[df_genero['timpo_total_jugado']==tiempo_maximo].index
resultado=df_genero.loc[indice_tiempo_maximo,:]

In [173]:
df_usuario=df_funcion2[(df_funcion2['user_id']=='shinomegami') & (df_funcion2['genero']=='Action')]

In [174]:
df_usuario

,genero,user_id,año_posted_review,timpo_total_jugado
3976423,Action,shinomegami,2014.0,8392
3976443,Action,shinomegami,2015.0,530882
3976520,Action,shinomegami,2012.0,137029


In [175]:
def UserForGenre( genero : str ): #Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.
    try:
        genero=genero.capitalize() # independientemente de si el usuario coloca el parametro en mayuscula o minuscula, el texto introducido se convierte a un texto con la primer letra en mayusculas y las otrwas en minusculas
        df_genero=df_funcion2[df_funcion2['genero']==genero]
        tiempo_maximo=df_genero['timpo_total_jugado'].max()
        indice_tiempo_maximo=df_genero[df_genero['timpo_total_jugado']==tiempo_maximo].index
        resultado=df_genero['user_id'].loc[indice_tiempo_maximo].values[0]

        df_horas_por_año=df_funcion2[(df_funcion2['user_id']==resultado) & (df_funcion2['genero']==genero)]
        lista=[]
        for indice,fila in df_horas_por_año.iterrows():
            año=fila['año_posted_review']
            horas_jugadas=fila['timpo_total_jugado']
            lista.append({'Año': año, 'Horas': horas_jugadas})

        return {f'Usuario con más horas jugadas para el Género {genero}':resultado, 'Horas jugadas': lista}

    except Exception as e:
        return {'Genero incorrecto'}

# Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}



In [176]:
UserForGenre('actIon')

{'Usuario con más horas jugadas para el Género Action': 'shinomegami',
 'Horas jugadas': [{'Año': 2014.0, 'Horas': 8392},
  {'Año': 2015.0, 'Horas': 530882},
  {'Año': 2012.0, 'Horas': 137029}]}

##### **Funcion 3** ______________________________________________________________________________________________________________________________________________________________________________

In [177]:
df_funcion3=pd.merge(df_user_reviews,df_user_items,on=['user_id','id_juego'],how='left').merge(df_steam_games,on='id_juego',how='left')

In [178]:
df_funcion3.head(3)

,user_id,id_juego,helpful,recommend,sentiment_analysis,review_posted,año_posted_review,items_count,nombre_juego,timpo_total_jugado,tiempo_jugado_ultimas_2semanas,publisher,genero,title,fecha_lanzamiento,price,early_access,developer,año_lanzamiento
0,76561197970982479,1250,No ratings yet,True,2,2011-11-05,2011,277.0,Killing Floor,10006.0,0.0,Tripwire Interactive,Action,Killing Floor,2009-05-14,19.99,False,Tripwire Interactive,2009.0
1,76561197970982479,22200,No ratings yet,True,2,2011-07-15,2011,277.0,Zeno Clash,271.0,0.0,ACE Team,Action,Zeno Clash,2009-04-21,9.99,False,ACE Team,2009.0
2,76561197970982479,22200,No ratings yet,True,2,2011-07-15,2011,277.0,Zeno Clash,271.0,0.0,ACE Team,Indie,Zeno Clash,2009-04-21,9.99,False,ACE Team,2009.0


In [179]:
df_funcion3=df_funcion3[(df_funcion3['recommend']==True) & ((df_funcion3['sentiment_analysis']==2) | (df_funcion3['sentiment_analysis']==1))]

In [180]:
df_funcion3=df_funcion3.loc[:,['id_juego','title','año_posted_review','recommend']]

In [181]:
df_funcion3

,id_juego,title,año_posted_review,recommend
0,1250,Killing Floor,2011,True
1,22200,Zeno Clash,2011,True
2,22200,Zeno Clash,2011,True
3,43110,NaN,2011,True
4,251610,NaN,2014,True
...,...,...,...,...
107589,730,Counter-Strike: Global Offensive,2015,True
107590,730,Counter-Strike: Global Offensive,2015,True
107591,253980,Enclave,2015,True
107593,730,Counter-Strike: Global Offensive,2015,True


In [182]:
df_funcion3=df_funcion3.groupby(by=['id_juego','title','año_posted_review']).count()

In [183]:
df_funcion3.reset_index(inplace=True)

In [184]:
df_funcion3

,id_juego,title,año_posted_review,recommend
0,10,Counter-Strike,2011,1
1,10,Counter-Strike,2012,3
2,10,Counter-Strike,2013,7
3,10,Counter-Strike,2014,18
4,10,Counter-Strike,2015,15
...,...,...,...,...
3314,419070,Warhammer: End Times - Vermintide Sigmar's Ble...,2015,6
3315,421630,A Study in Steampunk: Choice by Gaslight,2015,2
3316,423120,Community College Hero: Trial by Fire,2015,2
3317,423880,Carpe Diem,2015,27


In [185]:
df_funcion3=df_funcion3.loc[:,['año_posted_review','id_juego','title','recommend']]  # reordear data set a fines de mejor lectura 

In [186]:
df_funcion3.sort_values(by='año_posted_review',ascending=True)  # EXPLORACION , trabajar con este dataset, tomarlo como definitivo

,año_posted_review,id_juego,title,recommend
429,2010,13570,Tom Clancy's Splinter Cell Chaos Theory®,1
810,2010,39690,ArcaniA,1
620,2010,24010,Train Simulator,1
608,2010,22600,Worms Reloaded,2
32,2010,240,Counter-Strike: Source,2
...,...,...,...,...
1888,2015,233450,Prison Architect,105
1883,2015,233290,Murdered: Soul Suspect,6
1881,2015,233270,Far Cry 3 - Blood Dragon,2
1875,2015,233130,Shadow Warrior,12


In [187]:
# Exportar dataframe a csv con el que voy a trabajar definitivamente la funcion 3

df_funcion3.to_csv('funcion3.csv',index=False)

In [188]:
df_funcion3.head(7)

,año_posted_review,id_juego,title,recommend
0,2011,10,Counter-Strike,1
1,2012,10,Counter-Strike,3
2,2013,10,Counter-Strike,7
3,2014,10,Counter-Strike,18
4,2015,10,Counter-Strike,15
5,2014,20,Team Fortress Classic,4
6,2015,20,Team Fortress Classic,5


In [189]:
df_funcion3[df_funcion3['año_posted_review']==2013].nlargest(3,'recommend')



,año_posted_review,id_juego,title,recommend
69,2013,440,Team Fortress 2,1410
187,2013,4000,Garry's Mod,548
1174,2013,105600,Terraria,508


In [190]:
def UsersRecommend( año : int ): # Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
    try:
        df_top3=df_funcion3[df_funcion3['año_posted_review']==año].nlargest(3,'recommend')
        return [{'Puesto 1': df_top3['title'].iloc[0]},{'Puesto 2:': df_top3['title'].iloc[1]},{'Puesto 3:': df_top3['title'].iloc[2]}]

    except Exception:
        return {'No existen datos para el valor ingresado'}
    
#Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [191]:
UsersRecommend(2015)

[{'Puesto 1': 'Counter-Strike: Global Offensive'},
 {'Puesto 2:': 'Unturned'},
 {'Puesto 3:': 'Team Fortress 2'}]

##### **Funcion 4** ______________________________________________________________________________________________________________________________________________________________________________

In [192]:
df_funcion4=pd.merge(df_user_reviews,df_user_items,on=['user_id','id_juego'],how='left').merge(df_steam_games,on='id_juego',how='left')

In [193]:
df_funcion4=df_funcion4[(df_funcion4['recommend']==False) & (df_funcion4['sentiment_analysis']==0)]

In [194]:
df_funcion4=df_funcion4.loc[:,['año_posted_review','id_juego','title','recommend']]

In [195]:
df_funcion4=df_funcion4.groupby(by=['año_posted_review','id_juego','title']).count()

In [196]:
df_funcion4.reset_index(inplace=True)

In [197]:
df_funcion4  # data listao para hacer la funcion , deberia eliminar los nulos y revisar un poco 

,año_posted_review,id_juego,title,recommend
0,2011,440,Team Fortress 2,2
1,2011,18700,And Yet It Moves,4
2,2011,33460,From Dust,2
3,2011,63940,Men of War: Vietnam,1
4,2012,440,Team Fortress 2,2
...,...,...,...,...
815,2015,409510,Genesis Online,2
816,2015,410340,Liftoff,5
817,2015,412400,GASP,6
818,2015,417860,Emily is Away,20


In [198]:
df_funcion4.to_csv('funcion4.csv',index=False) # pasar a csv 

In [199]:
df_funcion4.head(5)

,año_posted_review,id_juego,title,recommend
0,2011,440,Team Fortress 2,2
1,2011,18700,And Yet It Moves,4
2,2011,33460,From Dust,2
3,2011,63940,Men of War: Vietnam,1
4,2012,440,Team Fortress 2,2


In [200]:
def UsersNotRecommend( año : int ): # Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
    try:
        df_top3=df_funcion3[df_funcion3['año_posted_review']==año].nsmallest(3,'recommend')
        return [{'Puesto 1': df_top3['title'].iloc[0]},{'Puesto 2:': df_top3['title'].iloc[1]},{'Puesto 3:': df_top3['title'].iloc[2]}]

    except Exception:
        return {'No existen datos para el valor ingresado'}

# Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [201]:
UsersNotRecommend(2013)

[{'Puesto 1': 'Day of Defeat'},
 {'Puesto 2:': 'Half-Life: Opposing Force'},
 {'Puesto 3:': 'Ricochet'}]

##### **Funcion 5** ______________________________________________________________________________________________________________________________________________________________________________

In [202]:
df_steam_games.head(3)


,publisher,genero,title,fecha_lanzamiento,price,early_access,id_juego,developer,año_lanzamiento
0,Kotoshiro,Action,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018
1,Kotoshiro,Casual,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018
2,Kotoshiro,Indie,Lost Summoner Kitty,2018-01-04,4.99,False,761140,Kotoshiro,2018


In [203]:
df_user_items.head(3)


,user_id,items_count,id_juego,nombre_juego,timpo_total_jugado,tiempo_jugado_ultimas_2semanas
0,76561197970982479,277,10,Counter-Strike,6,0
1,76561197970982479,277,20,Team Fortress Classic,0,0
2,76561197970982479,277,30,Day of Defeat,7,0


In [204]:
df_user_reviews.head(3)

,user_id,id_juego,helpful,recommend,sentiment_analysis,review_posted,año_posted_review
0,76561197970982479,1250,No ratings yet,True,2,2011-11-05,2011
1,76561197970982479,22200,No ratings yet,True,2,2011-07-15,2011
2,76561197970982479,43110,No ratings yet,True,2,2011-04-21,2011


In [205]:
df_funcion5= pd.merge(df_user_reviews,df_steam_games,on='id_juego',how='left')

In [206]:
df_funcion5=df_funcion5.loc[:,['user_id','id_juego','año_lanzamiento','sentiment_analysis']]

In [207]:
df_funcion5.dropna(inplace=True)

In [208]:
df_funcion5.to_csv('funcion5.csv',index=False)

In [209]:
df_funcion5

,user_id,id_juego,año_lanzamiento,sentiment_analysis
0,76561197970982479,1250,2009.0,2
1,76561197970982479,22200,2009.0,2
2,76561197970982479,22200,2009.0,2
5,js41637,227300,2013.0,0
6,js41637,227300,2013.0,0
...,...,...,...,...
107588,76561198239215706,730,2012.0,1
107589,wayfeng,730,2012.0,1
107590,76561198251004808,253980,2003.0,2
107591,72947282842,730,2012.0,0


In [210]:
def sentiment_analysis( año : int ): # Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
    df_año=df_funcion5[df_funcion5['año_lanzamiento']==año]
    positivos=0
    negativos=0
    neutros=0
    for i in df_año['sentiment_analysis'].values:
        if i==0:
            negativos+=1
        elif i==1:
            neutros+=1
        elif i==2:
            positivos+=1

    return {'Negative': negativos, 'Neutral': neutros,'Positive':positivos}

# Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [211]:
sentiment_analysis(2009)

{'Negative': 470, 'Neutral': 485, 'Positive': 1218}